In [ ]:
import unet
import image_processor
import torch
import os
import zipfile
from datetime import datetime
import torchvision
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
def modelWriter(model, path : str):
    torch.save(model.state_dict(), path)
    
#thanks chatGPT
def zip_file(file_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(file_path, arcname=os.path.basename(file_path))
    os.remove(file_path)


In [ ]:

imageDir = r""
labelDir = r""
modelSaveDir = r""



#class name
baseModel = "unet"

#downscale
downscale = 16

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((int(256/downscale), int(1024/downscale)))])

batchSizes = [16]
epochs = [100]
learningRates = [.001]

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

saveDir= f'{modelSaveDir}/{baseModel}/'



In [ ]:
for e in epochs:
    for lr in learningRates:
        for bs in batchSizes:
            model = unet.unet()
            model.to(device)

            trainLoader, valLoader = image_processor.imageDirsToLoaders(imageDir=imageDir, labelDir= labelDir, batchSize= bs, labelTransforms=transform, imageTransforms=transform)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            loss = torch.nn.MSELoss()

            print(f'EPOCHS: {e} | LR: {lr} | BATCH: {bs}')

            for i in range(e):
                #if (i+1) > 200:
                #    lr = .0001
                model.train()
                currentTime = datetime.now()
                formatTime = currentTime.strftime("%Y-%m-%d_%H-%M-%S")
                print(f'EPOCH: {i+1} | {formatTime}')
                runningLoss = 0
                trainingLoss = 0
                validationLoss = 0
                for image, label in trainLoader:
                    image = image.to(device)
                    label = label.to(device)
                    optimizer.zero_grad()
                    outputClasses = model(image)
                    
                    batchLoss = loss(outputClasses, label)
                    
                    batchLoss.backward()
                    optimizer.step()
                    trainingLoss += batchLoss

                currentTime = datetime.now()
                formatTime = currentTime.strftime("%Y-%m-%d_%H-%M-%S")
                trainingLoss = round(trainingLoss.item()/len(trainLoader))
                print(f"EPOCH: {i+1} | TRAIN LOSS: {trainingLoss} | {formatTime}")

                model.eval()
                with torch.no_grad():
                    for image, label in valLoader:
                        image = image.float()
                        label = label.float()
                        image = image.to(device)
                        label = label.to(device)
                        outputs = model(image)
                        valBatchLoss = loss(outputs, label)
                        validationLoss += valBatchLoss
                
                validationLoss = round(validationLoss.item()/len(valLoader))
                model.validationLosses.append(validationLoss)
                model.trainingLosses.append(trainingLoss)

                print(f"EPOCH: {i+1} | VAL LOSS: {validationLoss} | {formatTime}")

                detailedName = f'{baseModel}{downscale}x_VL_{validationLoss}_CE_{i+1}_TE_{e}_LR_{lr}_BS_{bs}_TS_{formatTime}'
                model.name = detailedName
                if not os.path.exists(saveDir):
                    os.makedirs(saveDir)
                
                modelSavePath = os.path.join(saveDir, f'{detailedName}.pt')
                
                model.saveAttributes(modelSavePath)
                
                print(f'{modelSavePath} saved')
                epochsSoFar = list(j for j in range(1, i+2))
                plt.plot(epochsSoFar, model.trainingLosses,  label = 'Training Loss', color = 'red')
                plt.plot( epochsSoFar, model.validationLosses, label = 'Validation Loss', color = 'blue')
                plt.legend()
                plt.show()

                if (i+1) % 10 == 0:
                    clear_output(wait=True)

            
            


